In [1]:
import requests
from bs4 import BeautifulSoup
import xlrd
import pandas as pd
import unidecode

In [2]:
def get_columns(df):
    """
    """
    create_new_df_for_columns = []
    three_count = 0
    counter = 0
    while three_count != 3:
        elements = list(df.iloc[counter])
        new_list = []
        important_element = ''
        for val in elements[:]:
            if val == '':
                important_element = important_element
            else:
                important_element = val
            new_list.append(important_element)
        lenghts = len(new_list) - new_list.count('')
        if lenghts:
            three_count += 1
            create_new_df_for_columns.append(new_list)
        else:
            three_count=0
        counter +=1
    col_df = pd.DataFrame(create_new_df_for_columns[:-2])
    for col in col_df.columns:
        col_df[col] = col_df[col].astype('str')
    columns = list(col_df.apply(lambda x: x + ' ').sum())
        
    return columns


def clean_text(txt):
    """
    """
    txt = txt.lower()
    txt = unidecode.unidecode(txt)
    txt = txt.replace('\n', ' ')
    txt = txt.replace('  ', ' ')
    txt = txt.strip(' ')
    txt = txt.replace(' ', '_')
    return txt

def clean_dataframe(df):
    """
    """
    new_columns = pd.Series(get_columns(df))
    df.columns = new_columns.apply(clean_text)
    nan_in_cols = df.replace('', None).isna().sum()
    cols = nan_in_cols[nan_in_cols <= 10*nan_in_cols.median()].index
    df = df[cols]
    nan_in_rows = df.replace('', None).T.isna().sum()
    rows = nan_in_rows[nan_in_rows <= 10*nan_in_rows.median()].index
    df = df.loc[rows].reset_index(drop=True)
    df = df.iloc[1:].reset_index(drop=True)
    return df

def read_workbook(path):
    """
    """
    xl_info = requests.get(path)
    workbook = xlrd.open_workbook(file_contents=xl_info.content)
    return workbook

In [3]:
# URL deseada para obtener la información:
url = 'https://www.gob.mx/cnbv/acciones-y-programas/bases-de-datos-de-inclusion-financiera'
# Obtener una respuesta.
html = requests.get(url)
if not html.ok:
    print('Response is between 400 and 600.', html.reason)
# Obtener el contenido
html_content = html.content
# Hacer una sopa bonita para buscar contenido de manera sencilla
soup = BeautifulSoup(html_content)

link_list = []
# En este caso los hipervínculos de interés están en a
for a_element in soup.find_all('a'):
    try:
        'Agarramos esos hipervínculos'
        link = a_element.__getitem__('href')
        link_list.append(link)
    except KeyError:
        print('element does not contain href')
        
excel_links = [excel_dataset_link for excel_dataset_link in link_list if 'xls' in excel_dataset_link]

element does not contain href
element does not contain href
element does not contain href


In [4]:
clean_info = []
raw_info = []
clean_dict = {}
not_info_as_expected = []
for excel_path in excel_links:
    workbook = read_workbook(excel_path)
    workbook_sheet_names = workbook.sheet_names()
    desired_sheet_set = [mun_sheet for mun_sheet in workbook_sheet_names if 'Mun' in mun_sheet]
    for desired_sheet in desired_sheet_set:
        if desired_sheet:
            df = pd.DataFrame(workbook.sheet_by_name(desired_sheet)._cell_values)
            raw_info.append(df)
            df = clean_dataframe(df.copy())
            name = excel_path.split('/')[-1]
            date = name.replace('%20', '_').split('_')[-1]
            date = date.split('.')[0]
            df['name'] = name
            df['sheet_name'] = desired_sheet
            df['url'] = excel_path
            df['date'] = date
            clean_info.append(df)
            if clean_dict.get(desired_sheet) is None:
                clean_dict[desired_sheet] = []
            else:
                clean_dict[desired_sheet].append(df)
            print(name, desired_sheet, sep='|')
        else:
            not_info_as_expected.append(excel_path)

Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Tenencia Uso Banca Mun
Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Tenencia Uso EACP Mun
Base_de_Datos_de_Inclusion_Financiera_202106.xlsm|BD Género Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Tenencia Uso Banca Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Tenencia Uso EACP Mun
Base_de_Datos_de_Inclusion_Financiera_202103.xlsm|BD Género Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Tenencia Uso Banca Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Tenencia Uso EACP Mun
Base_de_Datos_de_Inclusion_Financiera_202012.xlsm|BD Género Mun
Base_de_Datos_de_Inclusion_Financiera_202009.xlsm|BD Infraestructura Mun
Base_de_Datos_de_Inclusion_Financiera_202009.xlsm|BD Tenencia U

Base%20de%20Datos%20IF%20201106.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201106.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201106.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201103.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201103.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201103.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201012.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201012.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201012.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201009.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201009.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201009.xlsx|BD Uso EACP Mun
Base%20de%20Datos%20IF%20201006.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201006.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20201003.xlsx|BD Acceso Mun
Base%20de%20Datos%20IF%20201003.xlsx|BD Uso Mun
Base%20de%20Datos%20IF%20200912.xlsx|Municipios


In [5]:
clean_dict.keys()

dict_keys(['BD Infraestructura Mun', 'BD Tenencia Uso Banca Mun', 'BD Tenencia Uso EACP Mun', 'BD Género Mun', 'BD Acceso Mun', 'BD Uso Banca Mun', 'BD Uso EACP Mun', 'BD Uso Mun', 'Municipios'])

In [85]:
bd_infraestructura_mun = pd.concat(clean_dict['BD Infraestructura Mun'])

In [12]:
bd_tendencia_uso_banca_mun = pd.concat(clean_dict['BD Tenencia Uso Banca Mun'])

In [14]:
bd_tendencia_uso_eacp_mun = pd.concat(clean_dict['BD Tenencia Uso EACP Mun'])

In [15]:
bd_genero_mun = pd.concat(clean_dict['BD Género Mun'])

In [22]:
def get_unique_id(df):
    clave = df.informacion_general_clave_municipio.astype('int').astype('str')
    fecha = df.date.astype('str').apply(lambda x: x)
    df['clave_unica'] = clave + '_' + fecha
    return df

In [72]:
bd_tendencia_uso_banca_mun = get_unique_id(bd_tendencia_uso_banca_mun)
bd_tendencia_uso_eacp_mun = get_unique_id(bd_tendencia_uso_eacp_mun)
bd_infraestructura_mun = get_unique_id(bd_infraestructura_mun)
bd_genero_mun = get_unique_id(bd_genero_mun)

In [73]:
nan_gstuff = bd_genero_mun[bd_genero_mun.date.isin(['202103', '202003', '201903'])].isna().sum()
non_write_colsg = nan_gstuff[nan_gstuff>1].index
simple_bd_genero_mun = bd_genero_mun.drop(non_write_colsg, axis=1)

In [74]:
drop_cols = ['date',
 'indicadores_(transacciones_por_cada_10,000_adultos)_transacciones_en_cajeros_automaticos',
 'indicadores_(transacciones_por_cada_10,000_adultos)_transacciones_en_tpv',
 'indicadores_demograficos_(contratos_por_cada_10,000_adultos)_captacion_depositos_a_plazo',
 'indicadores_demograficos_(contratos_por_cada_10,000_adultos)_captacion_tarjetas_de_debito',
 'informacion_general_clave_estado',
 'informacion_general_clave_municipio',
 'informacion_general_estado',
 'informacion_general_municipio',
 'informacion_general_poblacion',
 'informacion_general_poblacion_adulta',
 'informacion_general_poblacion_adulta_hombres',
 'informacion_general_poblacion_adulta_mujeres',
 'informacion_general_region',
 'informacion_general_tipo_de_poblacion',
 'name',
 'sheet_name',
 'url']

drop_cols2 = [ 'date',
 'informacion_general_clave_estado',
 'informacion_general_clave_municipio',
 'informacion_general_estado',
 'informacion_general_municipio',
 'informacion_general_poblacion',
 'informacion_general_poblacion_adulta',
 'informacion_general_poblacion_adulta_hombres',
 'informacion_general_poblacion_adulta_mujeres',
 'informacion_general_region',
 'informacion_general_tipo_de_poblacion',
 'name',
 'sheet_name',
 'url']

In [75]:
general_data = bd_tendencia_uso_banca_mun.merge(
    bd_tendencia_uso_eacp_mun.drop(drop_cols, axis=1), on='clave_unica', how='left'
).merge(
    bd_infraestructura_mun.drop(drop_cols2, axis=1), on='clave_unica', how='left'
)
nan_stuff = general_data.isna().sum()
non_write_cols = nan_stuff[nan_stuff>1].index
general_data = general_data.drop(non_write_cols, axis=1)
simplified_data = general_data[general_data.clave_unica.str.endswith('03')]

In [79]:
drop_cols3 =[ 'date',
 'informacion_general_clave_estado',
 'informacion_general_clave_municipio',
 'informacion_general_estado',
 'informacion_general_municipio',
 'informacion_general_poblacion',
 'informacion_general_poblacion_adulta',
 'informacion_general_poblacion_adulta_hombres',
 'informacion_general_poblacion_adulta_mujeres',
 'informacion_general_region',
 'informacion_general_tipo_de_poblacion',
 'name',
 'sheet_name',
 'url'
]
simplified_data2 = simplified_data.merge(
    simple_bd_genero_mun.drop(drop_cols3, axis=1), 
    on='clave_unica',
    how='left')

In [80]:
simplified_data2.to_csv('./../rulo_mota_omath/data/raw/general_financial_inclusion.csv')